# सत्र ५ – मल्टी-एजंट ऑर्केस्ट्रेटर

Foundry Local वापरून संशोधक -> संपादक या दोन एजंटांच्या साध्या पाइपलाइनचे प्रात्यक्षिक.


### स्पष्टीकरण: अवलंबित्व स्थापना
`foundry-local-sdk` आणि `openai` स्थापित करते जे स्थानिक मॉडेल प्रवेश आणि चॅट पूर्णतेसाठी आवश्यक आहेत. पुनरावृत्तीक्षम.


# परिस्थिती
किमान दोन‑एजंट ऑर्केस्ट्रेटर पॅटर्न लागू करते:
- **संशोधक एजंट** संक्षिप्त तथ्यात्मक मुद्दे गोळा करतो
- **संपादक एजंट** कार्यकारी स्पष्टतेसाठी पुन्हा लिहितो

प्रत्येक एजंटसाठी सामायिक मेमरी, मध्यम आउटपुटचे अनुक्रमिक हस्तांतरण, आणि एक साधी पाइपलाइन फंक्शन दाखवते. अधिक भूमिका (उदा., समीक्षक, सत्यापनकर्ता) किंवा समांतर शाखांसाठी विस्तार करण्यायोग्य.

**पर्यावरणीय व्हेरिएबल्स:**
- `FOUNDRY_LOCAL_ALIAS` - वापरण्यासाठी डीफॉल्ट मॉडेल (डीफॉल्ट: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - प्राथमिक एजंट मॉडेल (ALIAS ला ओव्हरराइड करते)
- `AGENT_MODEL_EDITOR` - संपादक एजंट मॉडेल (डीफॉल्ट: प्राथमिक)

**SDK संदर्भ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**हे कसे कार्य करते:**
1. **FoundryLocalManager** स्वयंचलितपणे Foundry Local सेवा सुरू करते
2. निर्दिष्ट मॉडेल डाउनलोड करते आणि लोड करते (किंवा कॅश केलेला आवृत्ती वापरते)
3. संवादासाठी OpenAI-सुसंगत एंडपॉइंट प्रदान करते
4. प्रत्येक एजंट विशेष कामांसाठी वेगळे मॉडेल वापरू शकतो
5. अंगभूत पुनर्प्रयत्न लॉजिक तात्पुरत्या अपयशांना सहजतेने हाताळते

**महत्त्वाच्या वैशिष्ट्ये:**
- ✅ स्वयंचलित सेवा शोध आणि प्रारंभ
- ✅ मॉडेल जीवनचक्र व्यवस्थापन (डाउनलोड, कॅश, लोड)
- ✅ परिचित API साठी OpenAI SDK सुसंगतता
- ✅ एजंट विशेषीकरणासाठी मल्टी-मॉडेल समर्थन
- ✅ पुनर्प्रयत्न लॉजिकसह मजबूत त्रुटी हाताळणी
- ✅ स्थानिक अनुमान (क्लाउड API आवश्यक नाही)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### स्पष्टीकरण: मुख्य आयात आणि टायपिंग
एजंट संदेश संचयनासाठी dataclasses सादर करतो आणि स्पष्टतेसाठी टायपिंग सूचनांचा वापर करतो. पुढील एजंट क्रियांसाठी Foundry Local व्यवस्थापक आणि OpenAI क्लायंट आयात करतो.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### स्पष्टीकरण: मॉडेल प्रारंभ (SDK पॅटर्न)
Foundry Local Python SDK चा वापर मजबूत मॉडेल व्यवस्थापनासाठी:
- **FoundryLocalManager(alias)** - सेवा आपोआप सुरू करते आणि alias द्वारे मॉडेल लोड करते
- **get_model_info(alias)** - alias ला ठोस मॉडेल ID मध्ये रूपांतरित करते
- **manager.endpoint** - OpenAI क्लायंटसाठी सेवा endpoint प्रदान करते
- **manager.api_key** - API की प्रदान करते (स्थानिक वापरासाठी पर्यायी)
- वेगवेगळ्या एजंटसाठी स्वतंत्र मॉडेल्सना समर्थन देते (प्राथमिक vs संपादक)
- लवचिकतेसाठी एक्सपोनेंशियल बॅकऑफसह अंगभूत पुनर्प्रयत्न युक्ती
- सेवा तयार आहे याची खात्री करण्यासाठी कनेक्शन सत्यापन

**महत्त्वाचा SDK पॅटर्न:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**जीवनचक्र व्यवस्थापन:**
- योग्य साफसफाईसाठी व्यवस्थापक जागतिक स्तरावर संग्रहित केले जातात
- प्रत्येक एजंट विशेषीकरणासाठी वेगळ्या मॉडेलचा वापर करू शकतो
- स्वयंचलित सेवा शोध आणि कनेक्शन हाताळणी
- अपयशांवर एक्सपोनेंशियल बॅकऑफसह सौम्य पुनर्प्रयत्न

हे एजंट ऑर्केस्ट्रेशन सुरू होण्यापूर्वी योग्य प्रारंभ सुनिश्चित करते.

**संदर्भ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### स्पष्टीकरण: Agent आणि Memory वर्ग
`AgentMsg` नावाचा हलका वर्ग मेमरी नोंदींसाठी आणि `Agent` जो समाविष्ट करतो:
- **सिस्टम भूमिका** - Agent चा व्यक्तिमत्व आणि सूचना
- **संदेश इतिहास** - संभाषणाचा संदर्भ जपतो
- **act() पद्धत** - योग्य त्रुटी हाताळणीसह क्रिया पार पाडतो

Agent वेगवेगळ्या मॉडेल्सचा (प्राथमिक विरुद्ध संपादक) वापर करू शकतो आणि प्रत्येक Agent साठी स्वतंत्र संदर्भ राखतो. या पद्धतीमुळे शक्य होते:
- क्रियांच्या दरम्यान मेमरी टिकवणे
- प्रत्येक Agent साठी लवचिक मॉडेल नियुक्ती
- त्रुटींचे पृथक्करण आणि पुनर्प्राप्ती
- सुलभ साखळीकरण आणि समन्वय


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### स्पष्टीकरण: समन्वित पाइपलाइन
दोन विशेष एजंट तयार करते:
- **संशोधक**: प्राथमिक मॉडेल वापरतो, तथ्यात्मक माहिती गोळा करतो
- **संपादक**: स्वतंत्र मॉडेल वापरू शकतो (जर कॉन्फिगर केले असेल), सुधारणा करतो आणि पुन्हा लिहितो

`pipeline` फंक्शन:
1. संशोधक कच्ची माहिती गोळा करतो
2. संपादक कार्यकारी-तयार आउटपुटमध्ये सुधारणा करतो
3. दोन्ही मध्यम आणि अंतिम परिणाम परत करते

या पद्धतीमुळे शक्य होते:
- मॉडेल विशेषज्ञता (वेगवेगळ्या भूमिकांसाठी वेगवेगळे मॉडेल)
- बहु-स्तरीय प्रक्रिया करून गुणवत्ता सुधारणा
- माहिती परिवर्तनाची मागोवा घेण्याची क्षमता
- अधिक एजंट्स किंवा समांतर प्रक्रिया सहजपणे जोडण्याची सुविधा


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### स्पष्टीकरण: पाइपलाइन कार्यप्रदर्शन आणि निकाल
अनुपालन + विलंब विषयक प्रश्नावर मल्टी-एजंट पाइपलाइन कार्यान्वित करते, ज्यामध्ये दाखवले जाते:
- बहु-स्तरीय माहिती परिवर्तन
- एजंटची विशेषता आणि सहकार्य
- परिष्करणाद्वारे आउटपुट गुणवत्ता सुधारणा
- ट्रेसिबिलिटी (मध्यवर्ती आणि अंतिम आउटपुट संरक्षित)

**निकाल संरचना:**
- `question` - मूळ वापरकर्ता प्रश्न
- `research` - कच्चा संशोधन आउटपुट (तथ्यात्मक बुलेट्स)
- `final` - परिष्कृत कार्यकारी सारांश
- `models` - प्रत्येक टप्प्यासाठी कोणते मॉडेल वापरले गेले

**विस्तार कल्पना:**
1. गुणवत्ता पुनरावलोकनासाठी एक Critic एजंट जोडा
2. विविध पैलूंवर संशोधनासाठी समांतर संशोधन एजंट अंमलात आणा
3. तथ्य तपासणीसाठी एक Verifier एजंट जोडा
4. वेगवेगळ्या जटिलता स्तरांसाठी वेगवेगळी मॉडेल्स वापरा
5. पुनरावृत्ती सुधारण्यासाठी अभिप्राय लूप्स अंमलात आणा


### प्रगत: कस्टम एजंट कॉन्फिगरेशन

एजंटचे वर्तन सानुकूलित करण्याचा प्रयत्न करा, प्रारंभिक सेल चालवण्यापूर्वी पर्यावरणीय व्हेरिएबल्स बदलून:

**उपलब्ध मॉडेल्स:**
- टर्मिनलमध्ये `foundry model ls` वापरून सर्व उपलब्ध मॉडेल्स पहा
- उदाहरणे: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, इत्यादी.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
